In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [3]:
# 포즈 객체와 그리기 객체 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
drawing_spec1 = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(0, 255, 0))
drawing_spec2 = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(0, 0, 255))

# 웹캠 설정
cap = cv2.VideoCapture(0)


def detect_pose(landmarks):
    thershold = 0.9

    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    
    hip_y = (left_hip.y + right_hip.y) / 2
    knee_y = (left_knee.y + right_knee.y) /2

    if hip_y < knee_y * thershold:
        return "Standing"
    else:
        return "Sitting"

with mp_pose.Pose(
    static_image_mode=False, # False 일 때는 비디오 스트림일 경우
    model_complexity=1, # 0, 1, 2 의 순서로 정확도 / 숫자가 올라갈수록 정확도 상승 / 속도는 저하
    smooth_landmarks=True,  # 프레임간의 랜드마크 위치를 부드럽게 처리할 지 경정
    enable_segmentation=False, # 포즈 검출과 인물 마스크를 생성할지 여부 결정 -> 배경제거
    smooth_segmentation=True, # 마스크 경계를 부드럽게 처리할지 여부
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:

    # 레퍼런스 이미지 -> 자세이미지
    reference_img_path = "reference_pose.jpg"
    ref_img = cv2.imread(reference_img_path)
    ref_img_rgb = cv2.cvtColor(ref_img, cv2.COLOR_BGR2RGB)
    ref_pose_results = pose.process(ref_img_rgb)

    # 레퍼런스 이미지의 랜드마크를 저장
    ref_landmarks = None
    if ref_pose_results.pose_landmarks:
        ref_landmarks = ref_pose_results.pose_landmarks

    # 레퍼런스 이미지와의 유사도 계산
    def difference_pose(landmarks1, landmarks2):
        threshold = 0.2
        if landmarks1 is None or landmarks2 is None:
            return "Error"
        differences = []

        for lm1, lm2 in zip(landmarks1.landmark, landmarks2.landmark):
            difference = np.sqrt((lm1.x - lm2.x) **2 + (lm1.y - lm2.y) **2)
            differences.append(difference)

        result_diff = np.mean(differences)
        if result_diff < threshold:
            return "Good pose"
        else:
            return "Try pose"

    while cap.isOpened():
        res, image = cap.read()
        if not res:
            print('웹캠에서 이미지를 가져오는 것을 실패')
            break
        # bgr 이미지를 rgb 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)

        # 속도 개선을 위한 이미지 쓰기 불가능
        image.flags.writeable = False

        # 포즈 감지
        results = pose.process(image)

        # 이미지 다시 쓰기로 지정
        image.flags.writeable = True

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)
            # 전체 검출된 랜드마크
            pose_state = results.pose_landmarks

            # 포즈 검출 함수 적용
            detect_result = difference_pose(pose_state, ref_landmarks)

            cv2.putText(image, detect_result, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("frame", image)
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

c:\Users\Admin\anaconda3\envs\gumi_env310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
